In [10]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import pickle
import sys

In [11]:
def show_img(img):
    cv2.imshow('image',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.imwrite("./output/out.jpg", img)

In [12]:
scale_factor = 1.0
ref_white = cv2.resize(cv2.imread("images/aligned000.jpg", cv2.IMREAD_GRAYSCALE) / 255.0, (0,0), fx=scale_factor,fy=scale_factor)
ref_black = cv2.resize(cv2.imread("images/aligned001.jpg", cv2.IMREAD_GRAYSCALE) / 255.0, (0,0), fx=scale_factor,fy=scale_factor)
ref_avg   = (ref_white + ref_black) / 2.0
ref_on    = ref_avg + 0.05 # a threshold for ON pixels
ref_off   = ref_avg - 0.05 # add a small buffer region

In [13]:
h,w = ref_white.shape

# mask of pixels where there is projection
proj_mask = (ref_white > (ref_black + 0.05))

scan_bits = np.zeros((h,w), dtype=np.uint16)
proj_image = np.zeros((h,w,3), dtype=np.uint8)

In [14]:
#temp_image =0
def bool_img(x):
#show_img(proj_mask)
    t = x.astype(int)
    temp_image = np.zeros((h,w,1), dtype=np.uint8)
    temp_image[:,:,0] = t*255
    show_img(temp_image)
    cv2.imwrite("./output/out.jpg", temp_image)

In [15]:
 for i in range(0,15):
        # read the file
        patt = cv2.resize(cv2.imread("images/aligned%03d.jpg"%(i+2)), (0,0), fx=scale_factor,fy=scale_factor)
        patt_gray = cv2.resize(cv2.imread("images/aligned%03d.jpg"%(i+2), cv2.IMREAD_GRAYSCALE) / 255.0, (0,0), fx=scale_factor,fy=scale_factor)

        # mask where the pixels are ON
        on_mask = (patt_gray > ref_on) & proj_mask
        #bool_img(on_mask)
        # this code corresponds with the binary pattern code
        bit_code = np.uint16(1 << i)

        # TODO: populate scan_bits by putting the bit_code according to on_mask
        index = np.where(on_mask)
        for t in range(len(index[0])):
            i,j = index[0][t], index[1][t]
            scan_bits[i,j] = scan_bits[i,j] | bit_code
            #scan_bits[ on_mask] = scan_bits[on_mask] + bit_code

In [16]:
bool_img(scan_bits)

In [17]:
sum(sum(scan_bits>0))

257455

In [18]:
h,w

(960L, 1280L)

In [19]:
with open("binary_codes_ids_codebook.pckl","r") as f:
        binary_codes_ids_codebook = pickle.load(f)

camera_points = []
projector_points = []
for x in range(w):
    for y in range(h):
        if not proj_mask[y,x]:
            continue # no projection here
        if scan_bits[y,x] not in binary_codes_ids_codebook:
            continue # bad binary code

        x_p, y_p = binary_codes_ids_codebook[scan_bits[y,x]]
        if x_p >= 1279 or y_p >= 799: # filter
            continue
        camera_points.append([y/2.0,x/2.0])
        projector_points.append([y_p,x_p])
            # TODO: use binary_codes_ids_codebook[...] and scan_bits[y,x] to
            # TODO: find for the camera (x,y) the projector (p_x, p_y).
            # TODO: store your points in camera_points and projector_points

            # IMPORTANT!!! : due to differences in calibration and acquisition - divide the camera points by 2
        proj_image[y,x] = [0,y_p,x_p]
    # now that we have 2D-2D correspondances, we can triangulate 3D points!
    
show_img(proj_image)
#return
camera_points = np.asarray([camera_points], dtype=np.float32)
projector_points = np.array([projector_points], dtype=np.float32)
print(type(camera_points), camera_points.shape)
print(type(projector_points), projector_points.shape)

(<type 'numpy.ndarray'>, (1L, 241133L, 2L))
(<type 'numpy.ndarray'>, (1L, 241133L, 2L))


In [20]:
# load the prepared stereo calibration between projector and camera
with open("stereo_calibration.pckl","r") as f:
    d = pickle.load(f)
    camera_K    = d['camera_K']
    camera_d    = d['camera_d']
    projector_K = d['projector_K']
    projector_d = d['projector_d']
    projector_R = d['projector_R']
    projector_t = d['projector_t']

    # TODO: use cv2.undistortPoints to get normalized points for camera, use camera_K and camera_d
    # TODO: use cv2.undistortPoints to get normalized points for projector, use projector_K and projector_d
#camera_points = np.zeros((1,10,2), dtype=np.float32)
#projector_points = np.zeros((1,10,2), dtype=np.float32)
camera_undistorted = cv2.undistortPoints(camera_points, camera_K, camera_d)
projector_undistorted = cv2.undistortPoints(projector_points, projector_K, projector_d)

I = np.identity(3)
z = np.zeros((3,1), dtype=np.float32)
P1 = np.concatenate((I,z ),axis=1)
P2 = np.concatenate((projector_R, projector_t),axis=1)

#RL, RR, P1, P2, _, _, _ = cv2.stereoRectify(camera_K, camera_d, projector_K, projector_d, ref_avg.shape[::-1], projector_R, projector_t, alpha=-1)

    # TODO: use cv2.triangulatePoints to triangulate the normalized points
homo_cord = cv2.triangulatePoints(P1,P2,camera_undistorted, projector_undistorted)
    # TODO: use cv2.convertPointsFromHomogeneous to get real 3D points
points_3d = cv2.convertPointsFromHomogeneous(homo_cord.T)
	# TODO: name the resulted 3D points as "points_3d"
#mask = (points_3d[:,:,2] > 200) & (points_3d[:,:,2] < 1400)
#points_3d = points_3d[mask]

In [21]:
#mask = (points_3d[:,:,2] > 200) & (points_3d[:,:,2] < 1400)
#points_3d = points_3d[mask]
points_3d.shape

(241133L, 1L, 3L)

In [22]:
points_3d_new = []
for i in range(points_3d.shape[0]):
    pt = points_3d[i][0]
    #print(pt)
    if pt[2]>200 and pt[2]<1400:
        points_3d_new.append([pt])

In [23]:
points_3d_new = np.asarray(points_3d_new)
points_3d_new.shape

(233408L, 1L, 3L)

In [94]:
print("write output point cloud")
print(points_3d.shape)
output_name = sys.argv[1] + "output.xyz"
with open(output_name,"w") as f:
    for p in points_3d:
        print(p)
        f.write("%d %d %d\n"%(p[0,0],p[0,1],p[0,2]))
    #return points_3d #, camera_points, projector_points

write output point cloud
(98500L, 3L)
[-148.2102356    35.26653671  925.72241211]


IndexError: too many indices for array

In [10]:
proj_image.shape

(960L, 1280L, 3L)

In [11]:
proj_image[959][1200]

array([0, 0, 0], dtype=uint8)

In [60]:
camera_K

array([[ 849.005049  ,    0.        ,  355.69365051],
       [   0.        ,  800.12432908,  147.20034655],
       [   0.        ,    0.        ,    1.        ]])

In [61]:
projector_K

array([[  3.00859116e+03,   0.00000000e+00,   5.25485417e+02],
       [  0.00000000e+00,   2.66722446e+03,   5.34628897e+02],
       [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00]])

In [68]:
np.zeros((3,1), dtype=np.float32)

array([[ 0.],
       [ 0.],
       [ 0.]], dtype=float32)

In [75]:
I = np.identity(3)
z = np.zeros((3,1), dtype=np.float32)
np.concatenate((I,z ),axis=1)

array([[ 1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.]])

In [76]:
P2 = np.concatenate((projector_R, projector_t),axis=1)

In [77]:
P2

array([[ -9.98023725e-01,  -6.50097215e-03,  -6.25010484e-02,
         -1.68515114e+01],
       [  2.07499323e-03,  -9.97501121e-01,   7.06201679e-02,
          7.21346242e+01],
       [ -6.28039656e-02,   7.03509138e-02,   9.95543274e-01,
          5.38563319e+00]])